# Tutorial to run ONNX model in seldon-core

In order to run the tutorial you need to install the following packages:
- [minikube](https://github.com/kubernetes/minikube): local kubernetes cluster
- [helm](https://helm.sh/): package manager for kubernetes
- [s2i](https://github.com/openshift/source-to-image): source-to-image (s2i) - build containers from templates and source code.

Create the local kubernetes cluster.

In [1]:
!minikube start --memory 4096 --cpus 4

😄  minikube v1.3.1 on Darwin 10.14.3
🔥  Creating virtualbox VM (CPUs=4, Memory=4096MB, Disk=20000MB) ...
🐳  Preparing Kubernetes v1.15.2 on Docker 18.09.8 ...
🚜  Pulling images ...
🚀  Launching Kubernetes ... 
⌛  Waiting for: apiserver proxy etcd scheduler controller dns
🏄  Done! kubectl is now configured to use "minikube"


Create a kubernetes admin role.

In [2]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


Install helm.

In [3]:
!helm init

$HELM_HOME has been configured at /Users/naxty/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation


In [4]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


Add and install the seldcon operator.

In [5]:
!helm repo add seldon https://storage.googleapis.com/seldon-charts

"seldon" has been added to your repositories


In [6]:
!helm install seldon/seldon-core-operator --version 0.4.0

NAME:   peddling-dolphin
LAST DEPLOYED: Tue Sep  3 08:11:57 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  1s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  1s

==> v1/ConfigMap
NAME           DATA  AGE
seldon-config  1     1s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         1s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     1s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.105.185.120  <none>       443/TCP  1s
webhook-server-service                      ClusterIP  10.107.170.139  <none>       443/TCP  1s

==> v1/ServiceAc

In [7]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager

partitioned roll out complete: 1 new pods have been updated...


Install ambassador as ingress.

In [8]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Tue Sep  3 08:12:21 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-7cb6588db7-gjprf  0/1    ContainerCreating  0         0s
ambassador-7cb6588db7-tgw8p  0/1    ContainerCreating  0         0s
ambassador-7cb6588db7-vlhxl  0/1    ContainerCreating  0         0s

==> v1/Service
NAME              TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)                     AGE
ambassador        LoadBalancer  10.110.50.126  <pending>    80:30809/TCP,443:31164/TCP  0s
ambassador-admin  ClusterIP     10.105.221.81  <none>       8877/TCP                    0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME             AGE
ambassador       0s
ambassador-crds  0s

==> v1beta1/ClusterRoleBinding
NAME      

In [9]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


Build the container based with source-to-image.

In [10]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python37:0.11 emotion-service:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl


You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


Create the emotion_service deployment based on the json definition.

In [ ]:
!kubectl create -f emotion_service_deployment.json

In [12]:
!kubectl get deployments

NAME                                                   READY   UP-TO-DATE   AVAILABLE   AGE
ambassador                                             3/3     3            3           4m24s
emotion-service-deployment-emotion-predictor-780e070   0/1     1            0           3s


In [13]:
!kubectl rollout status deploy/emotion-service-deployment-emotion-predictor-780e070

Waiting for deployment "emotion-service-deployment-emotion-predictor-780e070" rollout to finish: 0 of 1 updated replicas are available...
deployment "emotion-service-deployment-emotion-predictor-780e070" successfully rolled out


Retrieve the minikube ip and ambassador port

In [14]:
!minikube ip

192.168.99.100


In [16]:
!kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'

30809

Test the service with the [smile image](images/smile.jpg)

![smile image](images/smile.jpg)

Todo: Replace the ip and port with the output of above.

In [20]:
!curl -vX POST http://192.168.99.100:30809/seldon/default/seldon-emotion/api/v0.1/predictions -d @payload.json --header "Content-Type: application/json"

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 192.168.99.100...
* TCP_NODELAY set
* Connected to 192.168.99.100 (192.168.99.100) port 30809 (#0)
> POST /seldon/default/seldon-emotion/api/v0.1/predictions HTTP/1.1
> Host: 192.168.99.100:30809
> User-Agent: curl/7.54.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 14889
> Expect: 100-continue
> 
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
< HTTP/1.1 200 OK
< x-application-context: application:8081
< content-type: application/json;charset=utf-8
< content-length: 468
< date: Tue, 03 Sep 2019 08:07:43 GMT
< x-envoy-upstream-service-time: 99
< server: envoy
< 
{
  "meta": {
    "puid": "b3p9347qqbqb0ikd8bf71o4vtd",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "emotion-classifier": "emotion-service:0.1"
    },
    "metrics": []
  },
  "jsonData": {
    "anger": "0.00013194185",
    "contempt": "0.0011185444",
    "disgust": "4.5716013e-05",
    "f

# Credits
- ONNX ResNet Tutorial by seldon.io: https://docs.seldon.io/projects/seldon-core/en/latest/examples/onnx_resnet.html